In [7]:
render_dict = ['.', 'P', 'T', '#']
def render_maze(maze: list[list[float]]) -> list[str]:
    ret = []
    for x in maze:
        row = ""
        for y in x:
            row += render_dict[y]
        ret.append(row)
    return ret

In [31]:
from util.maze_generator import maze_generator

maze = maze_generator((10, 10))
render_maze(maze)

['##########',
 '##########',
 '##########',
 '##########',
 '#######..#',
 '######...#',
 '#######P..',
 '#######T..',
 '####......',
 '####.....#']